In [1]:
import struct
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 函数调用阶段

def load_dat_file(file_path):
    # 打开 dat 文件
    with open(file_path, "rb") as f:
        # 读取文件内容
        data = f.read()

    # 解析字节数据
    result = struct.unpack("<" + "d" * (len(data) // 8), data)

    return result

def save_dat_file(file_path, data):
    # 将数据转换为字节数据
    binary_data = struct.pack("<" + "d" * len(data), *data)

    # 保存为二进制dat文件
    with open(file_path, "wb") as f:
        f.write(binary_data)

def log_transformation(data):
    # 取对数
    log_data = np.log10(data)

    # 计算bin_size
    bin_size = log_data[1] - log_data[0]

    # 计算新的起始点和终点
    new_start = log_data[0] - bin_size / 2
    new_end = log_data[-1] + bin_size / 2

    # 根据新的起始点和终点生成新的对数域数据
    new_data = np.linspace(new_start, new_end, len(energy)+1)

    new_data = np.power(10, new_data)

    return new_data
    

## en转换代码

In [5]:
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000" # 可以使用windows格式
log_file_path = os.path.join(base_dir, "chose.log") # 仅仅使用chose文件

# 读取参数文件
with open(log_file_path, "r") as f:
    parameters = f.readlines()[3:]

for parameter in parameters:
    parameter = parameter.strip()
    parameter_folder = parameter
    energy_file_path = os.path.join(base_dir, parameter_folder, "calspec", "en.dat")

    # 检查文件是否存在
    if not os.path.exists(energy_file_path):
        print(f"File not found: {energy_file_path}")
        continue

    energy = load_dat_file(energy_file_path)

    # 对能量数据进行取对数并计算bin_size
    new_energy = log_transformation(energy)

    new_energy_file_path = os.path.join(base_dir, parameter_folder, "calspec", "Xspec_en.dat")

    save_dat_file(new_energy_file_path, new_energy)

 #   print(f"Processed file: {new_energy_file_path}")
    print(len(new_energy))
    print(len(energy))
 #   print(type(energy))
 #   print(type(new_energy))

1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000
1001
1000


## flux清理代码

In [7]:
# 新方法
import os
import numpy as np
import struct
import matplotlib.pyplot as plt

# --- 读取和保存函数 ---
def load_dat_file(file_path):
    with open(file_path, "rb") as f:
        data = f.read()
    num_items = len(data) // 8
    return np.array(struct.unpack("<" + "d" * num_items, data))

def save_dat_file(file_path, data):
    with open(file_path, "wb") as f:
        f.write(struct.pack("<" + "d" * len(data), *data))

# --- 清理孤立非零段：左右 window 个点都为 0 才清理 ---
def clean_isolated_peaks_by_segment(flux, window=3):
    flux = np.array(flux, copy=True)
    n = len(flux)
    mask = flux != 0

    segments = []
    i = 0
    while i < n:
        if mask[i]:
            start = i
            while i + 1 < n and mask[i + 1]:
                i += 1
            end = i
            segments.append((start, end))
        i += 1

    for start, end in segments:
        left_block = flux[max(0, start - window):start]
        right_block = flux[end + 1:min(n, end + 1 + window)]

        if len(left_block) == window and len(right_block) == window:
            if np.all(left_block == 0) and np.all(right_block == 0):
                flux[start:end + 1] = 0

    return flux

# --- 主逻辑 ---
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000"
log_file_path = os.path.join(base_dir, "chose.log")

# 指定清理窗口大小
WINDOW = 1

# 读取参数列表（跳过前 3 行）
with open(log_file_path, "r") as f:
    parameters = f.readlines()[3:]

for param in parameters:
    param = param.strip()
    calspec_path = os.path.join(base_dir, param, "calspec")
    flux_path = os.path.join(calspec_path, "flux.dat")
    en_path = os.path.join(calspec_path, "en.dat")
    de_path = os.path.join(calspec_path, "de.dat")

    if not os.path.exists(flux_path) or not os.path.exists(en_path) or not os.path.exists(de_path):
        print(f"跳过缺失文件: {param}")
        continue

    flux = load_dat_file(flux_path)
    en = load_dat_file(en_path)
    de = load_dat_file(de_path)

    flux_clean = clean_isolated_peaks_by_segment(flux, window=WINDOW)

    # 保存结果
    save_path = os.path.join(calspec_path, "flux_clean.dat")
    save_dat_file(save_path, flux_clean)
    print(f"✓ Saved: {save_path}")


✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.000\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.100\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.200\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.300\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.400\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000\te_0.100_tau_10.500\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\wa

In [27]:
# 旧方法，清洗力度没那么大
import os
import numpy as np
import struct

# --- 读取和保存函数 ---
def load_dat_file(file_path):
    with open(file_path, "rb") as f:
        data = f.read()
    num_items = len(data) // 8
    return np.array(struct.unpack("<" + "d" * num_items, data))

def save_dat_file(file_path, data):
    with open(file_path, "wb") as f:
        f.write(struct.pack("<" + "d" * len(data), *data))

# --- 孤立值清理函数 ---
def clean_isolated_peaks_by_segment(flux, window=3):
    """
    清除被左右全为 0 包围的连续非零片段

    Parameters:
        flux (np.ndarray): 原始 flux 数组
        window (int): 左右窗口大小

    Returns:
        cleaned_flux (np.ndarray): 清洗后的 flux 数组（等长）
    """
    flux = np.array(flux, copy=True)
    n = len(flux)
    mask = flux != 0

    # 找出非零段
    segments = []
    i = 0
    while i < n:
        if mask[i]:
            start = i
            while i + 1 < n and mask[i + 1]:
                i += 1
            end = i
            segments.append((start, end))
        i += 1

    for start, end in segments:
        left_block = flux[max(0, start - window):start]
        right_block = flux[end+1:min(n, end+1 + window)]

        # 判断左右各 window 个点是否全为 0
        if len(left_block) == window and len(right_block) == window:
            if np.all(left_block == 0) and np.all(right_block == 0):
                flux[start:end+1] = 0

    return flux




# --- 主批处理逻辑 ---
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec"
log_file_path = os.path.join(base_dir, "chose.log")

# 读取参数列表
with open(log_file_path, "r") as f:
    parameters = f.readlines()[3:]

# 遍历每个参数目录
for param in parameters:
    param = param.strip()
    calspec_path = os.path.join(base_dir, param, "calspec")
    flux_path = os.path.join(calspec_path, "flux.dat")

    if not os.path.exists(flux_path):
        print(f"跳过缺失文件: {flux_path}")
        continue

    flux = load_dat_file(flux_path)
    flux_clean = clean_isolated_peaks_by_segment(flux, window=2)
    

    save_path = os.path.join(calspec_path, "flux_clean.dat")
    save_dat_file(save_path, flux_clean)
    print(f"✓ Saved: {save_path}")


✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.000\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.100\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.200\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.300\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.400\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec\te_0.100_tau_10.500\calspec\flux_clean.dat
✓ Saved: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6

## 平滑化代码

In [9]:
import os
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from skimage.restoration import denoise_tv_chambolle
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

# === 工具函数 ===
def load_dat_file(file_path):
    """读取二进制 .dat 文件，返回 double 类型的 np.ndarray"""
    with open(file_path, "rb") as f:
        return np.frombuffer(f.read(), dtype="<d")

def save_dat_file(file_path, data):
    """保存 np.ndarray 到 .dat 文件（double 类型）"""
    with open(file_path, "wb") as f:
        f.write(np.array(data, dtype="<d").tobytes())

# === 各种平滑方法 ===
def smooth_tv(flux, weight=0.2):
    return denoise_tv_chambolle(flux, weight=weight)

def smooth_lowess(flux, en, frac=0.06):
    return lowess(flux, en, frac=frac, return_sorted=False)

def smooth_rolling(flux, window=15):
    pad = np.pad(flux, (window//2, window//2), mode='edge')
    return np.convolve(pad, np.ones(window)/window, mode='valid')

def smooth_savgol(flux, window=51, polyorder=3):
    return savgol_filter(flux, window_length=window, polyorder=polyorder)

def smooth_gaussian(flux, sigma=4):
    return gaussian_filter1d(flux, sigma=sigma)

# === 批量处理路径 ===
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000"
log_file = os.path.join(base_dir, "chose.log")

with open(log_file, "r") as f:
    parameters = f.readlines()[3:]  # 跳过前三行空白行

for param in parameters:
    param = param.strip()
    folder = os.path.join(base_dir, param, "calspec")
    if not os.path.isdir(folder):
        print(f"❌ Skipping missing folder: {folder}")
        continue

    try:
        en = load_dat_file(os.path.join(folder, "en.dat"))
        flux = load_dat_file(os.path.join(folder, "flux.dat"))
    except Exception as e:
        print(f"⚠️ Error reading {param}: {e}")
        continue

    # === 各种平滑 ===
    try:
        save_dat_file(os.path.join(folder, "flux_smoothed_lowess.dat"), smooth_lowess(flux, en))
        save_dat_file(os.path.join(folder, "flux_smoothed_rolling.dat"), smooth_rolling(flux))
        save_dat_file(os.path.join(folder, "flux_smoothed_tv.dat"), smooth_tv(flux))
        save_dat_file(os.path.join(folder, "flux_smoothed_savgol.dat"), smooth_savgol(flux))
        save_dat_file(os.path.join(folder, "flux_smoothed_gaussian.dat"), smooth_gaussian(flux))
        print(f"✓ {param}: all 5 smoothings saved.")
    except Exception as e:
        print(f"❌ {param} smoothing error: {e}")


✓ te_0.100_tau_10.000: all 5 smoothings saved.
✓ te_0.100_tau_10.100: all 5 smoothings saved.
✓ te_0.100_tau_10.200: all 5 smoothings saved.
✓ te_0.100_tau_10.300: all 5 smoothings saved.
✓ te_0.100_tau_10.400: all 5 smoothings saved.
✓ te_0.100_tau_10.500: all 5 smoothings saved.
✓ te_0.100_tau_10.600: all 5 smoothings saved.
✓ te_0.100_tau_10.700: all 5 smoothings saved.
✓ te_0.100_tau_10.800: all 5 smoothings saved.
✓ te_0.100_tau_10.900: all 5 smoothings saved.
✓ te_0.100_tau_11.000: all 5 smoothings saved.
✓ te_0.100_tau_11.100: all 5 smoothings saved.
✓ te_0.100_tau_11.200: all 5 smoothings saved.
✓ te_0.100_tau_11.300: all 5 smoothings saved.
✓ te_0.100_tau_11.400: all 5 smoothings saved.
✓ te_0.100_tau_11.500: all 5 smoothings saved.
✓ te_0.100_tau_11.600: all 5 smoothings saved.
✓ te_0.100_tau_11.700: all 5 smoothings saved.
✓ te_0.100_tau_11.800: all 5 smoothings saved.
✓ te_0.100_tau_11.900: all 5 smoothings saved.
✓ te_0.100_tau_12.000: all 5 smoothings saved.
✓ te_0.100_ta

#### clean之后的平滑

In [12]:
import os
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from skimage.restoration import denoise_tv_chambolle
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

# === 工具函数 ===
def load_dat_file(file_path):
    """读取二进制 .dat 文件，返回 double 类型的 np.ndarray"""
    with open(file_path, "rb") as f:
        return np.frombuffer(f.read(), dtype="<d")

def save_dat_file(file_path, data):
    """保存 np.ndarray 到 .dat 文件（double 类型）"""
    with open(file_path, "wb") as f:
        f.write(np.array(data, dtype="<d").tobytes())

# === 各种平滑方法 ===
def smooth_tv(flux, weight=0.2):
    return denoise_tv_chambolle(flux, weight=weight)

def smooth_lowess(flux, en, frac=0.06):
    return lowess(flux, en, frac=frac, return_sorted=False)

def smooth_rolling(flux, window=15):
    pad = np.pad(flux, (window//2, window//2), mode='edge')
    return np.convolve(pad, np.ones(window)/window, mode='valid')

def smooth_savgol(flux, window=51, polyorder=3):
    return savgol_filter(flux, window_length=window, polyorder=polyorder)

def smooth_gaussian(flux, sigma=4):
    return gaussian_filter1d(flux, sigma=sigma)

# === 批量处理路径 ===
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data_calspec_1000"
log_file = os.path.join(base_dir, "chose.log")

with open(log_file, "r") as f:
    parameters = f.readlines()[3:]  # 跳过前三行空白行

for param in parameters:
    param = param.strip()
    folder = os.path.join(base_dir, param, "calspec")
    if not os.path.isdir(folder):
        print(f"❌ Skipping missing folder: {folder}")
        continue

    try:
        en = load_dat_file(os.path.join(folder, "en.dat"))
        flux_clean = load_dat_file(os.path.join(folder, "flux_clean.dat"))
    except Exception as e:
        print(f"⚠️ Error reading {param}: {e}")
        continue

    # === 各种平滑 ===
    try:
        save_dat_file(os.path.join(folder, "flux_clean_smoothed_lowess.dat"), smooth_lowess(flux_clean, en))
        save_dat_file(os.path.join(folder, "flux_clean_smoothed_rolling.dat"), smooth_rolling(flux_clean))
        save_dat_file(os.path.join(folder, "flux_clean_smoothed_tv.dat"), smooth_tv(flux_clean))
        save_dat_file(os.path.join(folder, "flux_clean_smoothed_savgol.dat"), smooth_savgol(flux_clean))
        save_dat_file(os.path.join(folder, "flux_clean_smoothed_gaussian.dat"), smooth_gaussian(flux_clean))
        print(f"✓ {param}: flux_clean smoothed results saved.")
    except Exception as e:
        print(f"❌ {param} flux_clean smoothing error: {e}")


✓ te_0.100_tau_10.000: flux_clean smoothed results saved.
✓ te_0.100_tau_10.100: flux_clean smoothed results saved.
✓ te_0.100_tau_10.200: flux_clean smoothed results saved.
✓ te_0.100_tau_10.300: flux_clean smoothed results saved.
✓ te_0.100_tau_10.400: flux_clean smoothed results saved.
✓ te_0.100_tau_10.500: flux_clean smoothed results saved.
✓ te_0.100_tau_10.600: flux_clean smoothed results saved.
✓ te_0.100_tau_10.700: flux_clean smoothed results saved.
✓ te_0.100_tau_10.800: flux_clean smoothed results saved.
✓ te_0.100_tau_10.900: flux_clean smoothed results saved.
✓ te_0.100_tau_11.000: flux_clean smoothed results saved.
✓ te_0.100_tau_11.100: flux_clean smoothed results saved.
✓ te_0.100_tau_11.200: flux_clean smoothed results saved.
✓ te_0.100_tau_11.300: flux_clean smoothed results saved.
✓ te_0.100_tau_11.400: flux_clean smoothed results saved.
✓ te_0.100_tau_11.500: flux_clean smoothed results saved.
✓ te_0.100_tau_11.600: flux_clean smoothed results saved.
✓ te_0.100_tau

## 废弃代码

In [15]:
from statsmodels.nonparametric.smoothers_lowess import lowess
import os

# LOWESS 平滑函数
def lowess_filter(data, en, frac=0.1):
    return lowess(data, en, frac=frac)[:, 1]  # LOWESS 返回的是一个二维数组，取第二列

# 读取日志文件并处理每个参数
base_dir = r"\\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data"  # 可以使用windows格式
log_file_path = os.path.join(base_dir, "chose.log")

# 读取参数文件
with open(log_file_path, "r") as f:
    parameters = f.readlines()[3:]

for parameter in parameters:
    parameter = parameter.strip()
    parameter_folder = parameter
    
    # 获取能量文件路径 (已经存在，所以不需要读取 en.dat)
    energy_file_path = os.path.join(base_dir, parameter_folder, "calspec", "en.dat")
    
    # 读取已存在的对数变换后的能量数据
    en = load_dat_file(energy_file_path)
    flux_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux.dat")
    # 读取 flux 数据
    flux_data = load_dat_file(flux_file_path)

    # 进行 LOWESS 平滑
    flux_smoothed_lowess = lowess_filter(flux_data, en)

    # 保存平滑后的数据
    flux_smoothed_lowess_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux_smoothed_lowess.dat")
    save_dat_file(flux_smoothed_lowess_file_path, flux_smoothed_lowess)

    print(f"Smoothed flux (LOWESS) saved to: {flux_smoothed_lowess_file_path}")
    print(len(flux_data))
    print(len(flux_smoothed_lowess))
    print(type(flux_data))  # 原始 flux 数据
    print(type(flux_smoothed_lowess))  # 平滑后的 flux 数据



Smoothed flux (LOWESS) saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.000\calspec\flux_smoothed_lowess.dat
200
200
<class 'tuple'>
<class 'numpy.ndarray'>
Smoothed flux (LOWESS) saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.100\calspec\flux_smoothed_lowess.dat
200
200
<class 'tuple'>
<class 'numpy.ndarray'>
Smoothed flux (LOWESS) saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.200\calspec\flux_smoothed_lowess.dat
200
200
<class 'tuple'>
<class 'numpy.ndarray'>
Smoothed flux (LOWESS) saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.300\calspec\flux_smoothed_lowess.dat
200
200
<class 'tuple'>
<class 'numpy.ndarray'>
Smoothed flux (LOWESS) saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\tes

In [17]:
print(flux_data)
print(flux_smoothed_lowess)

(25.301012853439904, 24.914058104336153, 24.181707096684963, 23.132160243306682, 22.984072370251482, 22.183896296034703, 21.853437379495038, 20.507103535037814, 20.127279185478706, 19.287291532658614, 18.431269624676496, 17.75770830441434, 17.24175195543848, 16.255856883545203, 15.82711273954801, 15.021191969117181, 14.14911921121642, 13.392039396978706, 12.829125959549925, 12.098695298118605, 11.41900812587854, 10.747760193475528, 10.163879378179447, 9.519555117969768, 8.959716001826713, 8.492047330602649, 7.891332743070393, 7.5392240134658906, 6.951529758739486, 6.633636435796329, 6.136802759372139, 5.715536752495265, 5.38975665162686, 4.992130985775461, 4.619379671952581, 4.267528911173516, 3.961038356989697, 3.698975450432923, 3.4310916377156744, 3.19602690828398, 2.925736383713735, 2.7812103180648524, 2.540102056271562, 2.3490394378709496, 2.2013579797185767, 2.0213414130133573, 1.9002638833247425, 1.72630324896511, 1.5941650890368018, 1.4768677896415832, 1.3802674321676622, 1.246

In [15]:
from scipy.signal import butter, filtfilt
import os

# Butterworth Filter函数
def butter_lowpass(cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order)
    return filtfilt(b, a, data)

# 遍历 chose.log 文件中的参数路径
for parameter in parameters:
    parameter = parameter.strip()
    parameter_folder = parameter
    
    flux_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux.dat")
    
    # 检查 flux.dat 文件是否存在
    if not os.path.exists(flux_file_path):
        print(f"File not found: {flux_file_path}")
        continue
    
    # 读取 flux 数据
    flux_data = load_dat_file(flux_file_path)
    
    # 进行 Butterworth 滤波平滑
    fs = 1  # 假设每个点之间的采样率为1（单位是Hz）
    cutoff = 0.1  # 设定低通滤波器的截止频率
    flux_smoothed = butter_filter(flux_data, cutoff, fs)
    
    # 保存平滑后的数据
    flux_smoothed_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux_smoothed_butter.dat")
    save_dat_file(flux_smoothed_file_path, flux_smoothed)
    
    print(f"Smoothed flux saved to: {flux_smoothed_file_path}")


Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.000\calspec\flux_smoothed_butter.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.100\calspec\flux_smoothed_butter.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.200\calspec\flux_smoothed_butter.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.300\calspec\flux_smoothed_butter.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.400\calspec\flux_smoothed_butter.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.500\calspec\flux_smoothed_but

In [17]:
from scipy.signal import savgol_filter

# 遍历 chose.log 文件中的参数路径
for parameter in parameters:
    parameter = parameter.strip()
    parameter_folder = parameter
    
    flux_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux.dat")
    
    # 检查 flux.dat 文件是否存在
    if not os.path.exists(flux_file_path):
        print(f"File not found: {flux_file_path}")
        continue
    
    # 读取 flux 数据
    flux_data = load_dat_file(flux_file_path)
    
    # 进行 Savitzky-Golay 滤波平滑
    flux_smoothed = savgol_filter(flux_data, window_length=51, polyorder=3)
    
    # 保存平滑后的数据
    flux_smoothed_file_path = os.path.join(base_dir, parameter_folder, "calspec", "flux_smoothed_savgol.dat")
    save_dat_file(flux_smoothed_file_path, flux_smoothed)
    
    print(f"Smoothed flux saved to: {flux_smoothed_file_path}")


Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.000\calspec\flux_smoothed_savgol.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.100\calspec\flux_smoothed_savgol.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.200\calspec\flux_smoothed_savgol.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.300\calspec\flux_smoothed_savgol.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.400\calspec\flux_smoothed_savgol.dat
Smoothed flux saved to: \\wsl.localhost\Ubuntu-22.04\home\hdw\data\monk\plot\warmcorona\test\test_smooth_10_6\data\te_0.100_tau_10.500\calspec\flux_smoothed_sav